<a href="https://colab.research.google.com/github/jyryu3161/biobigdata/blob/main/transcriptome_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### <b>edgeR package를 이용한 transcriptome 분석</b>
* normalized read count 사용

In [ ]:
# 패키지 설치
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("clusterProfiler")
BiocManager::install("enrichplot")
BiocManager::install("gprofiler2")
BiocManager::install("limma")
BiocManager::install("edgeR")
install.packages("pheatmap")
install.packages("ggplot2")
install.packages("ggrepel")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.19 (BiocManager 1.30.23), R 4.4.0 (2024-04-24)

Installing package(s) 'BiocVersion', 'clusterProfiler'

also installing the dependencies ‘zlibbioc’, ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘formatR’, ‘XVector’, ‘GenomeInfoDb’, ‘lambda.r’, ‘futile.options’, ‘gridGraphics’, ‘tweenr’, ‘polyclip’, ‘RcppEigen’, ‘gridExtra’, ‘RcppArmadillo’, ‘lazyeval’, ‘plogr’, ‘png’, ‘Biostrings’, ‘futile.logger’, ‘snow’, ‘BH’, ‘cowplot’, ‘fastmatch’, ‘ggplotify’, ‘patchwork’, ‘ggplot2’, ‘ggforce’, ‘ggrepel’, ‘viridis’, ‘tidygraph’, ‘graphlayouts’, ‘ape’, ‘tidytree’, ‘treeio’, ‘BiocGenerics’, ‘Biobase’, ‘IRanges’, ‘RSQLite’, ‘S4Vectors’, ‘KEGGREST’, ‘HDO.db’, ‘BiocParallel’, ‘fgsea’, ‘reshape2’, ‘aplot’, ‘ggfun’, ‘ggne

In [ ]:
# 1. 필요한 라이브러리를 불러옴
library(edgeR)
library(limma)
library(clusterProfiler)
library(enrichplot)
library(ggplot2)
library(gprofiler2)
library(pheatmap)
library(ggrepel)

# 2. 데이터를 불러옴
expr_data <- read.csv("transcriptome.csv", row.names=1)
expr_data <- expr_data[apply(expr_data, 1, var) != 0, ]
design <- read.csv("study_design.csv")

# 3. DEG 분석을 수행
group <- factor(design$condition) # study design 파일에서 group 정보를 불러옴
y <- DGEList(counts=expr_data, group=group)
y <- cpm(y)

# t-test
design_matrix <- model.matrix(~0+group) # design matrix 생성
fit <- lmFit(y, design_matrix)
contrast_matrix <- makeContrasts(Diff=grouptest-groupcontrol, levels=design_matrix) # group 이름 확인
fit2 <- contrasts.fit(fit, contrast_matrix)
fit2 <- eBayes(fit2)
ttest_res <- topTable(fit2, number=Inf)

# Multiple testing correction using FDR
ttest_res$adj.P.Val <- p.adjust(ttest_res$P.Value, method="fdr")

# 5. 결과를 시각화
# Volcano plot
deg_results <- data.frame(log2FoldChange = ttest_res$logFC, p_adj = ttest_res$adj.P.Val, Gene_name = rownames(res))
deg_results <- na.omit(deg_results)
Category <- c()
for(i in 1:length(deg_results$log2FoldChange)){
  if((deg_results$log2FoldChange[i] < -1) & (deg_results$p_adj[i] < 0.05)){
    Category[i] <- "Down-regulated"
  }else if((deg_results$log2FoldChange[i] > 1) & (deg_results$p_adj[i] < 0.05)){
    Category[i] <- "Up-regulated"
  }else{
    Category[i] <- "None"
  }
}
deg_results["Category"] <- Category
deg_results$Category <- factor(deg_results$Category,
                               levels = c("Down-regulated", "Up-regulated", "None"),
                               labels = c("blue", "red", "grey"))

top10_genes <- head(deg_results[order(deg_results$p_adj),], 10)
top10_gene_names <- top10_genes$Gene_name

my_plot_1 <- ggplot(data = deg_results, mapping = aes(x = log2FoldChange, y = -log10(p_adj))) +
  geom_point(aes(colour = Category), size = 2) +
  scale_color_identity(guide = 'legend', breaks = c("blue", "red"), labels = c("Down-regulated", "Up-regulated")) +
  geom_hline(yintercept = -log10(0.05), linetype = "dashed") +
  geom_vline(xintercept = c(-1, 1), linetype = "dashed") +
  labs(x = "log2 Fold Change", y = "-log10(p.adj)", colour = NULL) +
  ggtitle("Volcano Plot") +
  theme_minimal() +
  theme(plot.title = element_text(hjust = 0.5)) +
  geom_text_repel(data = top10_genes, aes(label = top10_gene_names), size = 3)

ggsave(filename = "./volcano plot.tiff", plot = my_plot_1, width = 8, height = 6, dpi = 300,
       device = tiff,
       bg = "white")
dev.off()

ggsave(filename = "./volcano plot.pdf", plot = my_plot_1, width = 8, height = 6, dpi = 300,
       device = pdf(),
       bg = "white")
dev.off()


# Heat map
top_genes <- rownames(head(ttest_res[order(ttest_res$adj.P.Val), ], 100)) # 상위 100개 유전자 선택
top_genes_data <- y[top_genes, ] # 상위 유전자에 대한 발현 데이터 선택

my_palette <- colorRampPalette(c("navy", "white", "red"))(50)
col_annotation <- design
rownames(col_annotation) <- col_annotation[,1]
col_annotation[ , 1] <- NULL
my_plot_2 <- pheatmap(top_genes_data, cluster_rows = TRUE, cluster_cols = TRUE, annotation_col = col_annotation,
                      scale = "row", color = my_palette, show_rownames = FALSE, show_colnames = FALSE, main = "Heat Map")

ggsave(filename = "./heatmap.tiff", plot = my_plot_2, width = 8, height = 6, dpi = 300,
       device = tiff)
dev.off()

ggsave(filename = "./heatmap.pdf", plot = my_plot_2, width = 8, height = 6, dpi = 300,
       device = pdf())
dev.off()


# PCA
pca_result <- prcomp(t(log2(y + 1)))
pca_df <- data.frame(PC1 = pca_result$x[, 1], PC2 = pca_result$x[, 2], Group = group)
my_plot_3 <- ggplot(pca_df, aes(x = PC1, y = PC2, color = Group)) + geom_point()

ggsave(filename = "./PCA.tiff", plot = my_plot_3, width = 8, height = 6, dpi = 300,
       device = tiff,
       type = "cairo")
dev.off()

ggsave(filename = "./PCA.pdf", plot = my_plot_3, width = 8, height = 6, dpi = 300,
       device = pdf(),
       bg = "white")
dev.off()

# 6. Enrichment Analysis
# get the significant up-regulated/down-regulated genes
up <- ttest_res[ttest_res$adj.P.Val < 0.05 & ttest_res$logFC > 1,]
down <- ttest_res[ttest_res$adj.P.Val < 0.05 & ttest_res$logFC < -1,]

# order genes by adjusted-pvalue
up_ordered = up[order(up$adj.P.Val, decreasing = FALSE),]
down_ordered = down[order(down$adj.P.Val, decreasing = FALSE),]

# ordered enrichment analysis
gp_up_ordered = gost(list("up-regulated" = row.names(up_ordered)), organism = "hsapiens", source = c("GO:BP", "GO:MF", "GO:CC", "KEGG"))
gp_down_ordered = gost(list("down-regulated" = row.names(down_ordered)), organism = "hsapiens", source = c("GO:BP", "GO:MF", "GO:CC", "KEGG"))

# Enrichment plot
top_BP_terms_up <- head(gp_up_ordered$result[gp_up_ordered$result$source == "GO:BP", "term_id"], 10)
top_MF_terms_up <- head(gp_up_ordered$result[gp_up_ordered$result$source == "GO:MF", "term_id"], 10)
top_CC_terms_up <- head(gp_up_ordered$result[gp_up_ordered$result$source == "GO:CC", "term_id"], 10)
top_KEGG_terms_up <- head(gp_up_ordered$result[gp_up_ordered$result$source == "KEGG", "term_id"], 10)
top_terms_up <- c(top_BP_terms_up, top_MF_terms_up, top_CC_terms_up, top_KEGG_terms_up)

top_BP_terms_down <- head(gp_down_ordered$result[gp_down_ordered$result$source == "GO:BP", "term_id"], 10)
top_MF_terms_down <- head(gp_down_ordered$result[gp_down_ordered$result$source == "GO:MF", "term_id"], 10)
top_CC_terms_down <- head(gp_down_ordered$result[gp_down_ordered$result$source == "GO:CC", "term_id"], 10)
top_KEGG_terms_down <- head(gp_down_ordered$result[gp_down_ordered$result$source == "KEGG", "term_id"], 10)
top_terms_down <- c(top_BP_terms_down, top_MF_terms_down, top_CC_terms_down, top_KEGG_terms_down)

p1 <- gostplot(gp_up_ordered, interactive = FALSE)
publish_gostplot(p1, highlight_terms = top_terms_up, width = 10, height = 20, filename = "upregulated_enrichment.pdf")

p2 <- gostplot(gp_down_ordered, interactive = FALSE)
publish_gostplot(p2, highlight_terms = top_terms_down, width = 10, height = 20, filename = "downregulated_enrichment.pdf")

# multi_gp_link <- gost(list("up-regulated" = row.names(up_ordered), "down-regulated" = row.names(down_ordered)),
#                      organism='hsapiens', source = c("GO:BP", "GO:MF", "GO:CC", "KEGG"), as_short_link = TRUE)


# 7. 결과를 csv 파일로 저장
write.csv(ttest_res, "ttest_DEG_results.csv")